In [438]:
import pandas as pd 

In [439]:
#Reading csv files and making copy of them
file = pd.read_json("D:\\datasets\\sarcasm_headlines\\Sarcasm_Headlines_Dataset_v2.json",lines=True)
df = file.copy()
#Getting some random rows from dataset
df.sample(10)

,is_sarcastic,headline,article_link
14505,1,authorities not even going to bother looking f...,https://www.theonion.com/authorities-not-even-...
26727,0,a commodore computer from the 1980s is still h...,https://www.huffingtonpost.com/entry/grand-rap...
1413,0,barack obama makes last-minute push to block s...,https://www.huffingtonpost.com/entry/barack-ob...
12742,1,nra spokesman: a hebrew?,https://www.theonion.com/nra-spokesman-a-hebre...
12246,1,area woman has more than 200 products to help ...,https://local.theonion.com/area-woman-has-more...
4928,0,here's what it's like to be somebody who hates...,https://www.huffingtonpost.com/entry/the-disgu...
17511,1,climatologists say humanity's best hope is hur...,https://www.theonion.com/climatologists-say-hu...
27733,1,woman launches into 4-minute self-deprecating ...,https://local.theonion.com/woman-launches-into...
24365,0,trump uses major policy speech to threaten to ...,https://www.huffingtonpost.com/entry/donald-tr...
15410,0,california's brutal drought could change the s...,https://www.huffingtonpost.com/entry/californi...


In [441]:
df = df.drop(['article_link'],axis=1)

In [442]:
df.is_sarcastic.value_counts(ascending=False)

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [443]:
#Getting inforamation of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
dtypes: int64(1), object(1)
memory usage: 447.3+ KB


In [444]:
#Extracting features
features = [text.lstrip() for text in df['headline'].to_numpy()]

In [445]:
features[:11]

['thirtysomething scientists unveil doomsday clock of hair loss',
 'dem rep. totally nails why congress is falling short on gender, racial equality',
 'eat your veggies: 9 deliciously different recipes',
 'inclement weather prevents liar from getting to work',
 "mother comes pretty close to using word 'streaming' correctly",
 'my white inheritance',
 '5 ways to file your taxes with less stress',
 "richard branson's global-warming donation nearly as much as cost of failed balloon trips",
 'shadow government getting too large to meet in marriott conference room b',
 'lots of parents know this scenario',
 'this lesbian is considered a father in indiana (and an amazing one at that)']

In [446]:
#Importing needed modules . 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token='<00U00>')

In [447]:
#Tokenizing texts
tokenizer.fit_on_texts(features)
#Converting them to sequneces
sequences = tokenizer.texts_to_sequences(features)

In [448]:
#Converting them to sequneces
matrix = pad_sequences(sequences,padding='post',truncating='post',maxlen=15)
#Finding longest sentence .
lengths = list()
for i in matrix :
    lengths.append(len(i))
print(max(lengths))

15


In [449]:
#Splitting train and valodation data .
idx = (85*len(matrix))//100
train = matrix[:idx]
train_label = df['is_sarcastic'].to_numpy().reshape(-1,1)[:idx]
valid = matrix[idx:]
valid_label = df['is_sarcastic'].to_numpy().reshape(-1,1)[idx:]

In [450]:
print(train.shape)
print(train_label.shape)
print(valid.shape)
print(valid_label.shape)

(24326, 15)
(24326, 1)
(4293, 15)
(4293, 1)


In [451]:
#Building our model by a function .
from tensorflow.keras import layers , models
from tensorflow.keras.optimizers import Adam
def Create_model()->None :
    '''function docstring'''
    model = models.Sequential([layers.Embedding(len(tokenizer.word_index)+1,16,input_length=15)
                               ,layers.Flatten()
                               ,layers.Dense(6,activation='relu')
                               ,layers.Dropout(0.2)
                              ,layers.Dense(1,activation='sigmoid')])
    model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    return model
print('Model has built')

Model has built


In [452]:
#Training datas with 2 epoch .
history = Create_model().fit(train,train_label,epochs=2,validation_data=(valid,valid_label))

Epoch 1/2
761/761 [==============================] - 10s 13ms/step - loss: 0.4712 - accuracy: 0.7751 - val_loss: 0.3125 - val_accuracy: 0.8668
Epoch 2/2
761/761 [==============================] - 9s 12ms/step - loss: 0.1959 - accuracy: 0.9283 - val_loss: 0.2970 - val_accuracy: 0.8719
